<a href="https://colab.research.google.com/github/ManarHelmy/OCT-retinal-disease/blob/main/OCT_eyes_diseases_diagnosis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **OCT: Optical Coherence Tomography - Eyes Diseases Diagnosis**


---

In [ ]:
!pip install Kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [ ]:
!kaggle datasets download -d paultimothymooney/kermany2018

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/kermany2018
License(s): CC-BY-NC-SA-4.0
100% 10.8G/10.8G [01:04<00:00, 103MB/s]
100% 10.8G/10.8G [01:04<00:00, 181MB/s]


In [ ]:
!unzip /content/kermany2018.zip

Streaming output truncated to the last 5000 lines.
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8050636-2.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055145-1.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055145-2.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055145-3.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055590-1.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055590-2.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055590-3.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-1.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-2.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-3.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-4.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-5.jpeg  
  inflating: 

In [ ]:
classes = {0 : 'CNV',
           1 : 'DME',
           2 : 'Drusen',
           3 : 'Normal'}

In [ ]:
classes[0]

'CNV'

importing libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
# Instead of 'from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Use tensorflow.keras for preprocessing
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout
# Instead of 'from keras.models import Sequential'
from tensorflow.keras.models import Sequential # Use tensorflow.keras for models
# Instead of 'from keras.layers import Dense,Input,Flatten'
from tensorflow.keras.layers import Dense,Input,Flatten # Use tensorflow.keras for layers
from tensorflow.keras.utils import load_img,img_to_array
from sklearn.metrics import confusion_matrix
from IPython import get_ipython
from IPython.display import display

# Reinstall numpy and pandas to resolve the compatibility issue
!pip install numpy --upgrade --force-reinstall
!pip install pandas --upgrade --force-reinstall

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import os
os.listdir('/content/OCT2017 /train')

['DME', 'NORMAL', 'DRUSEN', 'CNV']

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_data = train_datagen.flow_from_directory('/content/OCT2017 /train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset='training')
test_data = train_datagen.flow_from_directory('/content/OCT2017 /train',
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=False,
                                              subset='validation')

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
train_data.image_shape

In [ ]:
np.unique(test_data.labels)

In [ ]:
train_data.class_indices

---

# **Modeling**

## VGG 16 Architecture

In [ ]:
vgg16_model = vgg16.VGG16(include_top=False, input_shape=(224,224,3))
vgg16_model.summary()

In [ ]:
for layer in vgg16_model.layers[:]:
  layer.trainable = False
vgg16_model.summary()

In [ ]:
model = Sequential()
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))
model.summary()

In [ ]:
len(train_data.labels)

In [ ]:
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])

results = model.fit(train_data, epochs=30, validation_data=test_data,
                  steps_per_epoch=len(train_data), validation_steps=len(test_data),
                  callbacks=EarlyStopping(patience=5, monitor='val_accuracy', restore_best_weights=True)
                  )

In [ ]:
loss, acc = model.evaluate(test_data)

In [ ]:
print("VGG 16")
print('Test accuracy = ',round(acc*100,2),'%\nLoss = ',loss ,sep='')

In [ ]:
results.history['val_loss']

In [ ]:
results.history['val_accuracy']

In [ ]:
i = results.history['val_loss'].index(loss)
i

In [ ]:
results.history['val_accuracy'][i]

In [ ]:
fig = plt.figure()
plt.plot(results.history['loss'], c='red', label='loss')
plt.plot(results.history['accuracy'], c='blue', label='accuracy')
plt.scatter(i ,results.history['accuracy'][i], c='orange', label='choosen parameters', marker='o')
plt.title("Training Data")
plt.legend(loc="center left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(results.history['val_loss'], c='red', label='val loss')
plt.plot(results.history['val_accuracy'], c='blue', label='val accuracy')
plt.scatter(i ,results.history['val_accuracy'][i], c='orange', label='choosen parameters', marker='o')
plt.title("Validation Data")
plt.legend(loc="center left")
plt.show()

---

# Inception v3 Architecture
(best accuracy)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K
import math

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

x = base_model.output
x = AveragePooling2D()(x)
x = Dropout(.5)(x)
x = Flatten()(x)
x = Dense(4, kernel_initializer='glorot_uniform', kernel_regularizer=l2(.0005), activation='softmax')(x)
model2 = Model(inputs=base_model.input, outputs=x)
model2.summary()

In [ ]:
opt = SGD(lr=.1, momentum=.9)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
results2 = model2.fit(train_data, epochs=5, validation_data=test_data,
                  steps_per_epoch=len(train_data), validation_steps=len(test_data),
                  callbacks = EarlyStopping(patience=3, monitor='val_accuracy',restore_best_weights=True)
                  )

In [ ]:
loss2, acc2 = model2.evaluate(test_data)

In [ ]:
print("Inception v3")
print("Test Accuracy:",round(acc2*100,2),"%","\nTest Loss:",loss2)

In [ ]:
results2.history['val_loss']

In [ ]:
results2.history['val_accuracy']

In [ ]:
i = results2.history['val_loss'].index(loss2)
i

In [ ]:
results2.history['val_accuracy'][i]

In [ ]:
fig = plt.figure()
plt.plot(results2.history['loss'], c='red', label='loss')
plt.plot(results2.history['accuracy'], c='blue', label='accuracy')
plt.scatter(i ,results2.history['accuracy'][i], c='orange', label='choosen parameters', marker='o')
plt.title("Training Data")
plt.legend(loc="center left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(results2.history['val_loss'], c='red', label='val loss')
plt.plot(results2.history['val_accuracy'], c='blue', label='val accuracy')
plt.scatter(i ,results2.history['val_accuracy'][i], c='orange', label='choosen parameters', marker='o')
plt.title("Validation Data")
plt.legend(loc="center left")
plt.show()

---

In [ ]:
yp = model2.predict(test_data)
yp

In [ ]:
yp = yp.argmax(axis=1)
yp

In [ ]:
yp.shape

In [ ]:
cnv = load_img('/kaggle/input/kermany2018/OCT2017 /test/CNV/CNV-1016042-1.jpeg',target_size=(224,224))
plt.imshow(cnv)
plt.title('CNV Sample')

In [ ]:
dme = load_img('/kaggle/input/kermany2018/OCT2017 /test/DME/DME-1081406-1.jpeg',target_size=(224,224))
plt.imshow(dme)
plt.title('DME Sample')

In [ ]:
dru = load_img('/kaggle/input/kermany2018/OCT2017 /test/DRUSEN/DRUSEN-1083159-1.jpeg',target_size=(224,224))
plt.imshow(dru)
plt.title('Drusen Sample')

In [ ]:
n = load_img('/kaggle/input/kermany2018/OCT2017 /test/NORMAL/NORMAL-1017237-1.jpeg',target_size=(224,224))
plt.imshow(n)
plt.title('Normal Sample')

In [ ]:
fig = plt.figure(figsize=(14,7))
# plt.grid=False
fig.add_subplot(1, 4, 1)
plt.axis('off')
plt.imshow(cnv)
plt.title('CNV')
# _______________________________
fig.add_subplot(1, 4, 2)
plt.axis('off')
plt.imshow(dme)
plt.title('DME')
# _______________________________
fig.add_subplot(1, 4, 3)
plt.axis('off')
plt.imshow(dru)
plt.title('Drusen')
# _______________________________
fig.add_subplot(1, 4, 4)
plt.axis('off')
plt.imshow(n)
plt.title('Normal')

In [ ]:
cnv = img_to_array(cnv)
n = img_to_array(n)
dme = img_to_array(dme)
dru = img_to_array(dru)
cnv = cnv/255
n = n/255
dme = dme/255
dru = dru/255
cnv = cnv.reshape(1,224,224,3)
n = n.reshape(1,224,224,3)
dme = dme.reshape(1,224,224,3)
dru = dru.reshape(1,224,224,3)
cnv.shape

In [ ]:
p1 = model2.predict(cnv).argmax()
print("Actual: CNV")
print("Predicted: Class ",p1,": ",classes[p1],sep='')

In [ ]:
p1 = model2.predict(dme).argmax()
print("Actual: DME")
print("Predicted: Class ",p1,": ",classes[p1],sep='')

In [ ]:
p1 = model2.predict(dru).argmax()
print("Actual: Drusen")
print("Predicted: Class ",p1,": ",classes[p1],sep='')

In [ ]:
p1 = model2.predict(n).argmax()
print("Actual: Normal")
print("Predicted: Class ",p1,": ",classes[p1],sep='')

In [ ]:
df = pd.DataFrame({'filename':test_data.filenames,'y_actual':test_data.classes,'Y_pred':yp})
df

In [ ]:
test_data.labels

In [ ]:
m = confusion_matrix(test_data.labels,yp)
m

In [ ]:
print("Heatmap\n")
m = pd.crosstab(test_data.labels, yp, rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(m,annot=True, cmap='Blues')

---